In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import matplotlib.dates as mdates
import matplotlib.cm as cm
import statsmodels.api as sm
import os
import glob

In [ ]:
#USEFUL BUT DON'T RUN AGAIN UNTIL NECESSARY!!!!!!

from wwo_hist import retrieve_hist_data

frequency = 24
start_date = '1-JAN-2020'
end_date = '14-MAY-2020'
api_key = 'e72365de52ae4343a1185640200505'
location_list = ['alabama','alaska','arizona','arkansas','california',
                 'colorado','connecticut','delaware','florida','georgia',
                 'hawaii','idaho','illinois','indiana','iowa','kansas',
                 'kentucky','louisiana','maine','maryland','massachusetts',
                'michigan','minnesota','mississippi','missouri','montana',
                'nebraska','nevada','new-hampshire', 'new-jersey', 'new-mexico',
                'new-york', 'north-carolina','north-dakota','ohio','oklahoma',
                'oregon','pennsylvania','rhode-island','south-carolina',
                'south-dakota', 'tennessee', 'texas','utah','vermont','virginia',
                'washington','west-virginia','wisconsin','wyoming']
hist_weather_data = retrieve_hist_data(api_key,
                                location_list,
                                start_date,
                                end_date,
                                frequency,
                                location_label = False,
                                export_csv = True,
                                store_df = True)

In [ ]:
location_list = ['alabama','alaska','arizona','arkansas','california',
                 'colorado','connecticut','delaware','florida','georgia',
                 'hawaii','idaho','illinois','indiana','iowa','kansas',
                 'kentucky','louisiana','maine','maryland','massachusetts',
                'michigan','minnesota','mississippi','missouri','montana',
                'nebraska','nevada','new-hampshire', 'new-jersey', 'new-mexico',
                'new-york', 'north-carolina','north-dakota','ohio','oklahoma',
                'oregon','pennsylvania','rhode-island','south-carolina',
                'south-dakota', 'tennessee', 'texas','utah','vermont','virginia',
                'washington','west-virginia','wisconsin','wyoming']

In [ ]:
# Load in U.S. specific data
raw_states = pd.read_csv('USA_cases.csv')
# population
#https://github.com/camillol/cs424p3/blob/master/data/Population-Density%20By%20State.csv
raw_pop = pd.read_csv('popdensity.csv')

In [ ]:
# Clean state case data
US_data = raw_states.groupby(['Province_State']).sum()
us_data = US_data.drop(['FIPS','Lat','Long_','code3','UID'],axis=1)
us_data.columns = pd.to_datetime(us_data.columns)
us_data.index = us_data.index.str.lower()
us_data.index = us_data.index.str.replace(' ', '-')

In [ ]:
# clean pop data
pop_data = raw_pop.drop(["GEO.id","GEO.id2"],axis=1)
pop_data = pop_data.set_index(["GEO.display-label"])
pop_data.index = pop_data.index.str.lower()
pop_data.index = pop_data.index.str.replace(' ', '-')

In [ ]:
# Load in weather
def weather(locationL):
    dfs = []
    path = "./states/"
    for i in range(len(locationL)):
        df = pd.read_csv(path + locationL[i] +".csv")
        df = df[['date_time','tempC','humidity','precipMM','windspeedKmph']]
        df['date'] = pd.to_datetime(df['date_time'])
        df.set_index('date',drop=True,inplace=True)
        df = df.drop(['date_time'],axis=1)
        popden = pop_data["Density per square mile of land area"][i]
        #Add popn density column
        df['popden'] = [popden for i in range(len(df))]
        dfs.append(df)
    return dfs

In [ ]:
# Load in U.S. specific data

weather_dfs = weather(location_list)

In [ ]:
# state data
def state_cases(locationL):
    dfs = []
    for l in locationL:
        df = us_data.loc[str(l)]
        df = pd.DataFrame(df)
        df.columns = ['cases']
        dfs.append(df)
    return dfs

In [ ]:
# Load in U.S. specific data
case_dfs = state_cases(location_list)
case_dfs

In [ ]:
# Total datset merges by state
def mergeDF(casesL, weatherL):
    dfs = []
    for i in range(len(casesL)):
        df = casesL[i].merge(weatherL[i],left_index=True, right_index=True)
        dfs.append(df)
    return dfs

In [ ]:
mergedDFs = mergeDF(case_dfs,weather_dfs)

In [ ]:
def getR(mergedDFs):
    out = []
    for df in mergedDFs:
        df['I_n'] = df.cases.diff()
        df.loc[df.cases>30]
        rs = df.I_n / df.I_n.shift(1)
        df['R'] = rs
        df = df.replace([np.inf, -np.inf], np.nan)
        df = df.dropna()
        out.append(df)
    return out


In [ ]:
final_dfs = getR(mergedDFs)
colors = cm.rainbow(np.linspace(0, 1, 50))
final_dfs

In [ ]:
def plt_scatter(final_dfs,colors, element):
    for i in range(len(final_dfs)):
        plt.scatter(final_dfs[i][element], final_dfs[i].R, color=colors[i])
    plt.ylabel('R')
    plt.xlabel(element)
    plt.title('R  vs. ' + element)
    plt.show()

In [ ]:
plt_scatter(final_dfs,colors,'tempC')
plt_scatter(final_dfs,colors,'humidity')
plt_scatter(final_dfs,colors,'precipMM')
plt_scatter(final_dfs,colors,'windspeedKmph')

In [ ]:
def lin_reg(final_dfs, location_list, element):
    # Linear regression using scipy
    output = pd.DataFrame(columns=['p_value','R^2','slope'])
    for i in range(len(final_dfs)):
        slope, intercept, r_value, p_value, std_err = stats.linregress(final_dfs[i][element], final_dfs[i].R)
        Rsq = r_value**2
        output.loc[str(location_list[i])] = [p_value, Rsq , slope]
    return output

In [ ]:
temp = lin_reg(final_dfs,location_list,'tempC')
humid = lin_reg(final_dfs,location_list,'humidity')
rain = lin_reg(final_dfs,location_list,'precipMM')
wind = lin_reg(final_dfs,location_list,'windspeedKmph')
print(temp[temp.p_value<0.05])
print(humid[humid.p_value<0.05])
print(rain[rain.p_value<0.05])
print(wind[wind.p_value<0.05])

In [455]:
def ols(final_dfs, location_list):
    # Linear regression using scipy
    output = pd.DataFrame(columns=["Intercept","tempC",'precipMM','windspeedKmph','humidity','precipMM:windspeedKmph','precipMM:humidity','windspeedKmph:humidity','tempC:humidity','tempC:windspeedKmph','tempC:precipMM','popden'])
    coeffs = pd.DataFrame(columns=["Intercept","tempC",'precipMM','windspeedKmph','humidity','precipMM:windspeedKmph','precipMM:humidity','windspeedKmph:humidity','tempC:humidity','tempC:windspeedKmph','tempC:precipMM','popden'])

    for i in range(len(final_dfs)):
        model_lin = sm.OLS.from_formula("R~tempC+precipMM+windspeedKmph+humidity+precipMM:windspeedKmph+precipMM:humidity+windspeedKmph:humidity+tempC:humidity+tempC:windspeedKmph+tempC:precipMM+popden", data=final_dfs[i])
        result_lin = model_lin.fit()
        #print(result_lin.summary())
        #print(result_lin.params)
        output.loc[location_list[i]] = [i for i in result_lin.pvalues]
        coeffs.loc[location_list[i]] = [i for i in result_lin.params]
        #output.loc[str(location_list[i])] = [i for i in result_lin.params]
    return output, coeffs


In [463]:
olsout, coeffs = ols(final_dfs, location_list)
olsout = olsout[olsout<0.05]
olsout = olsout.dropna(how='all')

for i in olsout.index:
    print(coeffs.loc[i])



Intercept                -0.018997
tempC                     0.767247
precipMM                 -2.236770
windspeedKmph             2.011400
humidity                  1.143010
precipMM:windspeedKmph    0.085731
precipMM:humidity        -0.004666
windspeedKmph:humidity   -0.043902
tempC:humidity           -0.027718
tempC:windspeedKmph       0.067724
tempC:precipMM            0.063314
popden                   -1.063833
Name: arkansas, dtype: float64
Intercept                 0.000040
tempC                    -1.068331
precipMM                 -0.590428
windspeedKmph            -0.529436
humidity                 -0.247378
precipMM:windspeedKmph    0.001652
precipMM:humidity         0.008309
windspeedKmph:humidity    0.005669
tempC:humidity            0.012101
tempC:windspeedKmph       0.007977
tempC:precipMM           -0.011275
popden                    0.029779
Name: connecticut, dtype: float64
Intercept                   0.000033
tempC                    -109.200567
precipMM             

In [ ]:
def ols_p(final_dfs, location_list):
    # Linear regression using scipy
    output = pd.DataFrame(columns=["Intercept","tempC",'windspeedKmph','humidity','windspeedKmph:humidity','tempC:humidity','tempC:windspeedKmph','popden'])
    coeffs = pd.DataFrame(columns=["Intercept","tempC",'windspeedKmph','humidity','windspeedKmph:humidity','tempC:humidity','tempC:windspeedKmph','popden'])
    for i in range(len(final_dfs)):
        model_lin = sm.OLS.from_formula("R~tempC+windspeedKmph+humidity+windspeedKmph:humidity+tempC:humidity+tempC:windspeedKmph+popden", data=final_dfs[i])
        result_lin = model_lin.fit()
        output.loc[str(location_list[i])] = [i for i in result_lin.pvalues]
        #coeffs.loc[str(location_list[i])] = [i for i in result_lin.params]
        #print(result_lin.summary())
    return output, coeffs

In [448]:
out = ols_p(final_dfs, location_list)
out = out[out<0.05]

out = out.dropna(how='all')
out

,Intercept,tempC,windspeedKmph,humidity,windspeedKmph:humidity,tempC:humidity,tempC:windspeedKmph,popden
arizona,NaN,NaN,NaN,NaN,NaN,0.047227,NaN,NaN
arkansas,NaN,NaN,NaN,0.045851,NaN,NaN,NaN,NaN
connecticut,0.019324,NaN,NaN,NaN,NaN,NaN,NaN,0.019324
georgia,NaN,NaN,0.010021,NaN,0.00438,NaN,NaN,NaN
maine,0.000708,0.000553,0.014874,NaN,NaN,NaN,0.016417,0.000708
maryland,NaN,NaN,NaN,NaN,NaN,NaN,0.003534,NaN
north-carolina,0.014156,0.016443,NaN,NaN,NaN,NaN,0.000937,0.014156
wyoming,NaN,NaN,0.018437,NaN,NaN,NaN,NaN,NaN
